# GESIS-Daten herunterladen, bearbeiten und explorieren mit R <a class="tocSkip">

## Setup

Zunächst müssen die benötigten R-Pakete geladen werden.

In [3]:
library(tidyverse)
library(haven)
library(gesis)
library(ggthemes)
library(ggmap)

ERROR: Error in library(gesis): there is no package called 'gesis'


Um in R Daten aus dem [Datenbestandskatalog (DBK) von GESIS](https://dbk.gesis.org/dbksearch/?db=d) herunterladen zu können, muss ein Objekt mit Ihren Zugangsdaten für den DBK erzeugt werden. Hierzu müssen Sie in der nachfolgenden Code-Zelle die E-Mail-Adresse, die Sie für die Registrierung beim GESIS-DBK verwendet haben, sowie Ihr Passwort für den DBK-Account an den entsprechenden Stellen einfügen.

In [ ]:
s <- login(username = "", password = "")

Mit dem nachfolgenden Befehl können Sie die im GESIS-DBK verfügbaren Studiengruppen explorieren.

In [ ]:
get_study_groups()

In diesem Notebook arbeiten wir mit Daten aus der [Allgemeinen Bevölkerungsumfrage der Sozialwissenschaften (ALLBUS)](https://www.gesis.org/allbus/allbus/) sowie der [European Values Study (EVS)](https://europeanvaluesstudy.eu/). Mit den folgenden Befehlen können Sie schauen, welche Datensätze es für diese Umfrageprogramme im GESIS-DBK gibt.

In [ ]:
# ALLBUS
get_datasets("0007") %>%
    print(n = 100)
# EVS
get_datasets("0009") %>% 
    print(n = 100)

Der Ordnung halber legen wir nun noch einen eigenen (Unter-)Ordner für die Datensätze an.

In [ ]:
dir.create(file.path(".", "data"))

## ALLBUS 1980-2016

Im ersten Schritt müssen wir den Datensatz aus dem DBK herunterladen und in R einlesen. Im missverständlich benannten Argument doi der Funktion download_dataset muss die ZA-Studiennummer angegeben werden. Als Format wählen wir den SPSS-Datensatz aus (filetype = ".sav") und geben als Zweck (für diesen Workshop) Weiterbildung und Qualifikation an (purpose = 6). Generell muss die [Benutzungsordnung](https://www.gesis.org/fileadmin/upload/dienstleistung/daten/umfragedaten/_bgordnung_bestellen/2018-05-25_Benutzungsordnung_GESIS_DAS.pdf) beachtet werden. Daten der Kategorien A, B und C sind ausschließlich für die akademische Forschung und Lehre freigegeben. Hinweis aus der Benutzungsordnung (Version vom 25.05.2018): "Institute oder Personen außerhalb der akademischen Forschung und Lehre können eine Bereitstellung schriftlich beantragen." Dies geht am einfachsten per E-Mail an <datenservice.das@gesis.org>.

In [ ]:
download_dataset(s, doi = "4587", path = "./data", filetype = ".sav", purpose = 6)
allbus_kum <- read_sav("data/ZA4587_v1-0-0.sav")
dim(allbus_kum)
names(allbus_kum)

Der Datensatz entält 64684 Fälle (Befragte) und 1020 Variablen. Wenn man sich die Namen der Variablen anschaut, wird klar, dass das [Codebuch](https://dbk.gesis.org/dbksearch/download.asp?db=D&id=64444) benötigt wird, um diese zu verstehen (dieses kann man auch mit dem gesis-Paket für R über den Befehl download_codebook(doi = "4587") herunterladen).

Im nächsten Schritt wählen wir die Variablen aus, die uns für die nachfolgenden Analysen interessieren und bearbeiten diese so, dass wir sie später im gewünschten Format plotten können. Die ausgewählten Items konnten in der Befragung mit "Ja" oder "Nein" beantwortet werden. Um die Interpretierbarkeit sowie die Berechnung von prozentualen Anteilen zu erleichtern, codieren wir den ursprünglichen für die Antwort "Nein" genutzten Wert 2 in 0 um. Aufgrund der Umstellung der Stichprobenziehung müssen die Daten für die Zeitreihen gewichtet werden (siehe [Codebuch zur Studie](https://dbk.gesis.org/dbksearch/download.asp?db=D&id=64444), S. xxi - xxxvi). Da es in dieser exemplarischen Visualisierung um Aussagen über Personen im Ost-West-Vergleich geht, ist (nur) das Personengewicht nötig. Zur Gewichtung wird einfach der Variablenvektor mit dem entsprechenden Gewichtungsvektor multipliziert (wichtiger Hinweis: laut Codebuch, S. 1680 bedeutet der Wert 0 in der Gewichtungsvariable, dass diese nicht berechnet werden konnte). 

In [ ]:
contact <- allbus_kum %>% 
  select(year, eastwest, starts_with("mc"), wghtpt) %>% 
  mutate(eastwest = as_factor(eastwest),
         eastwest = recode(eastwest,
                           "ALTE BUNDESLAENDER" = "Westdeutschland",
                           "NEUE BUNDESLAENDER" = "Ostdeutschland"),
         wghtpt = na_if(wghtpt, 0),
         mc01 = recode(as.numeric(mc01), "2"= 0),
         mc02 = recode(as.numeric(mc02), "2"= 0),
         mc03 = recode(as.numeric(mc03), "2"= 0),
         mc04 = recode(as.numeric(mc04), "2"= 0),
         mc01_w = mc01 * wghtpt,
         mc02_w = mc02 * wghtpt,
         mc03_w = mc03 * wghtpt,
         mc04_w = mc04 * wghtpt)

Bevor wir die Zeitreihen plotten können, müssen die Daten noch vom Wide- ins Long-Format überführt werden. Hinweis: Da es in einigen Jahren Fälle gibt, für die kein Gewicht berechnet werden konnte, ändert sich für diese Jahre entsprechend die jeweilige Fallzahl, die für die Berechnung der prozentualen Anteile benötigt wird.

In [ ]:
contact_long <- contact %>%
  filter(!is.na(wghtpt)) %>% 
  select(year, eastwest, mc01_w:mc04_w, wghtpt) %>% 
  gather(key = type_of_contact, value = yes_no, -year, -eastwest, -wghtpt) %>%
  mutate(type_of_contact = recode(type_of_contact,
                                  "mc01_w" = "In der Familie",
                                  "mc02_w" = "Am Arbeitsplatz",
                                  "mc03_w" = "In der Nachbarschaft",
                                  "mc04_w" = "Im Freundeskreis")) %>%
  filter(!is.na(yes_no)) %>% 
  group_by(year, eastwest, type_of_contact) %>% 
  summarise(sum = sum(yes_no), n = sum(wghtpt)) %>% 
  mutate (Anteil = (sum/n)*100)

Um die X-Achse des Zeitreihenplots korrekt mit Labels versehen zu können, schauen wir, in welchen Jahren die Fragen, die uns hier interessieren, gestellt wurden.

In [ ]:
unique(contact_long$year)

Nun können wir die Zeitreihen zu den vier Fragen im Ost-West-Vergleich plotten.

In [ ]:
contact_long %>% 
  ggplot(aes(x = year, y = Anteil, color = eastwest, shape = eastwest)) +
  geom_point(size = 2) +
  geom_line() +
  scale_y_continuous(expand = c(0,0),
                     breaks = seq(0, 100, 10),
                     limits = c(0,100),
                     labels = c("0%", "10%", "20%", "30%", "40%" , "50%", "60%", "70%", "80%", "90%", "100%")) +
  scale_x_continuous(breaks = c(1980, 1984, 1988, 1990, 1994, 1996, 2000, 2002, 2006, 2010, 2012, 2016)) +
  facet_wrap(~type_of_contact, scales = "free") +
  labs(x = "",
       y = "Anteil der Befragten mit Kontakten",
       title = "Kontakte zwischen Deutschen und Ausländern 1980-2016\n",
       caption = "Quelle: ALLBUScompact - Kumulation 1980-2016.GESIS Datenarchiv, Köln.\nZA4587 Datenfile Version 1.0.0, doi:10.4232/1.13048") +
  theme(axis.text.x = element_text (angle = 90, vjust = 0.5),
        panel.grid.major.x = element_blank(),
        panel.grid.minor.x = element_blank(),
        legend.title = element_blank(),
        plot.title = element_text(face="bold", size = 12))

## ALLBUS 2016

Die folgenden Beispiele sollen illustrieren, warum Vergleiche zwischen Bundesländern mit den ALLBUS-Daten problematisch sind. Hierzu verwenden wir Daten aus dem ALLBUS 2016. Auch diese müssen zunächst heruntergeladen (in diesem Fall auch noch entpackt), eingelesen und entsprechend aufbereitet werden.

In [ ]:
download_dataset(s, doi = "5250", path = "./data", filetype = ".sav", purpose = 6)
unzip("./data/ZA5250_v2-1-0.sav.zip", exdir = "./data")
allbus_2016 <- read_sav("data/ZA5250_v2-1-0.sav")
dim(allbus_2016)
migration_att <- allbus_2016 %>% 
  select(eastwest, land, ma09, mp03) %>%
  mutate(eastwest = as_factor(eastwest), 
         land = as_factor(land),
         land = fct_recode(land, "ST" = "SACHSEN-ANHALT",
                           "SN" = "SACHSEN",
                           "NI" = "NIEDERSACHSEN",
                           "BW" = "BADEN-WUERTTEMBERG",
                           "HE" = "HESSEN",
                           "MV" = "MECKLENB.-VORPOMMERN",
                           "BE" = "EHEM. BERLIN-OST",
                           "BE" = "EHEM. BERLIN-WEST",
                           "BB" = "BRANDENBURG",
                           "RP" = "RHEINLAND-PFALZ",
                           "SL" = "SAARLAND",
                           "TH" = "THUERINGEN",
                           "BY" = "BAYERN",
                           "NW" = "NORDRHEIN-WESTFALEN",
                           "SH" = "SCHLESWIG-HOLSTEIN",
                           "HB" = "BREMEN",
                           "HH" = "HAMBURG"))

Da in diesen Beispielen Mittelwerte und Konfidenzintervalle visualisiert werden sollen, erstellen wir zu diesem Zweck eigene Dataframes mit den entsprechenden Summary Statistics für die beiden Beispielvariablen.

In [ ]:
fremd_land <- migration_att %>%
  group_by(land) %>%
  summarise(n=n(),
    mean=mean(ma09, na.rm = T),
    sd=sd(ma09, na.rm = T)) %>%
  mutate(se=sd/sqrt(n)) %>%
  mutate(ic=se * qt((1-0.05)/2 + .5, n-1))

fremd_land

bereicher_land <- migration_att %>%
  group_by(land) %>%
  summarise(n=n(),
            mean=mean(mp03, na.rm = T),
            sd=sd(mp03, na.rm = T)) %>%
  mutate(se=sd/sqrt(n)) %>%
  mutate(ic=se * qt((1-0.05)/2 + .5, n-1))

bereicher_land

Nun können wir die Mittelwerte + Konfidenzintervalle pro Bundesland für die beiden exemplarisch ausgewählten Variablen plotten.

In [ ]:
ggplot(fremd_land, aes(x = land, y = mean)) + 
  geom_point(aes(size = n)) +
  geom_errorbar(aes(ymin = mean-ic, ymax = mean+ic), width = .1) + 
  scale_y_continuous(expand = c(0,0),
                     limits = c(1,7)) +
  labs(x = "",
       y = "",
       title = "Durch die vielen Ausländer in Deutschland fühlt man sich zunehmend \nals Fremder im eigenen Land.",
       subtitle = "1 = Stimme überhaupt nicht zu, 7 = Stimme voll und ganz zu",
       caption = "Quelle: ALLBUS 2016. GESIS Datenarchiv, Köln. ZA5250 Datenfile Version 2.1.0, doi:10.4232/1.12796") +
  theme_calc()

ggplot(bereicher_land, aes(x = land, y = mean)) + 
  geom_point(aes(size = n)) +
  geom_errorbar(aes(ymin = mean-ic, ymax = mean+ic), width = .1) +
  scale_y_continuous(expand = c(0,0),
                     limits = c(1,7)) +
  labs(x = "",
       y = "",
       title = "Die in Deutschland lebenden Ausländer \nsind eine Bereicherung für die Kultur in Deutschland.",
       subtitle = "1 = Stimme überhaupt nicht zu, 7 = Stimme voll und ganz zu",
       caption = "Quelle: ALLBUS 2016. GESIS Datenarchiv, Köln. ZA5250 Datenfile Version 2.1.0, doi:10.4232/1.12796") + 
theme_calc()

Die beiden Plots zeigen, dass die Konfidenzintervalle für manche Bundesländer durch die geringe Stichprobengröße sehr groß sind (v.a. Bremen und Saarland), wodurch Vergleiche zwischen/mit diesen wenig aussagekräftig werden.

## EVS 2017

Die Daten aus der EVS 2017 sind derzeit (Stand 14.01.2019) nur als Pre-Release mit Daten aus 16 Ländern verfügbar. Auch diese müssen wir wieder herunterladen, entpacken, einlesen und aufbereiten (Variablen rekodieren, damit höhere Werte größere Zustimmung bedeuten).

In [ ]:
download_dataset(s, doi = "7500", path = "./data", filetype = ".sav", purpose = 6)
unzip("./data/ZA7500_v1-0-0.sav.zip", exdir = "./data")
evs_2017 <- read_sav("data/ZA7500_v1-0-0.sav")
dim(evs_2017)
pol_evs_2017 <- evs_2017 %>% 
  select(country, v124, v131, v142, v144) %>% 
  mutate(country = as_factor(country),
         v124 = recode(as.numeric(v124), "1" = 4,
                       "2" = 3,
                       "3" = 2,
                       "4" = 1),
         v131 = recode(as.numeric(v131), "1" = 4,
                       "2" = 3,
                       "3" = 2,
                       "4" = 1))

Da wir die Daten auf einer Karte darstellen wollen, müssen wir nun auch Weltkartendaten aus dem Package ggmap laden.

In [ ]:
world <- map_data("world")

Nun aggregieren wir die EVS-Daten auf Länderebene.

In [ ]:
pol_country_evs_2017 <- pol_evs_2017 %>% 
  group_by(country) %>% 
  summarize_all(mean, na.rm = T)

Bevor wir die Karten- mit den EVS-Daten verknüpfen können, müssen wir schauen, ob sich die Namen der Länder in den beiden Datensätzen unterscheiden, und die Länder dann entsprechend umbenennen.

In [ ]:
pol_country_evs_2017 %>% 
  anti_join(world, by = c("country" = "region"))
world %>% 
    select(region) %>% 
    unique()
pol_country_evs_2017 <- pol_country_evs_2017 %>% 
  mutate(country = recode(country, `Slovak Republic` = 'Slovakia'))

Nun können wir die Karten- und EVS-Daten miteinander verbinden.

In [ ]:
evs_map_2017 <- world %>% 
  left_join(pol_country_evs_2017, by = c("region" = "country"))

Um die Skalen für die Karten sinnvoll anpassen zu können, sollten wir uns die Deskriptivstatistiken für die Variablen anschauen.


In [ ]:
pol_country_evs_2017 %>% 
  select(-country) %>% 
  summary()

Hinweis: Zur Gewährleistung der Vergleichbarkeit mit den Plots für die EVS-Daten aus dem Jahr 2008 wurden auch deren Summary Statistics (siehe unten) bei der Auswahl der Skalen für die Plots berücksichtigt.

Im ersten Plot geht es ums Vertrauen in die EU im Ländervergleich.

In [ ]:
ggplot(data = evs_map_2017, aes(x = long, y = lat, group = group, fill = v124)) + 
  geom_polygon(color = "white") + 
  xlim(-25, 47.0) + ylim(35, 70) +
  theme_map() +
  theme(plot.title = element_text(face="bold", size = 16)) +
  labs(title = "Vertrauen in die EU 2017",
       subtitle = "1 = Überhaupt kein Vertrauen, 4 = Sehr viel Vertrauen",
       fill = "",
       caption = "Quelle: European Values Study 2017: Integrated Dataset (EVS 2017). \nGESIS Datenarchiv, Köln. ZA7500 Datenfile Version 1.0.0, doi:10.4232/1.13090") +
  scale_fill_distiller(breaks = c(1, 1.5, 2, 2.5, 3, 3.5, 4), palette = "RdYlGn", direction = 1, limits = c(1.5,3.5))

Nun vergleichen wir das Vertrauen in die Regierung.

In [ ]:
ggplot(data = evs_map_2017, aes(x = long, y = lat, group = group, fill = v131)) + 
  geom_polygon(color = "white") + 
  xlim(-25, 47.0) + ylim(35, 70) +
  theme_map() +
  theme(plot.title = element_text(face="bold", size = 16)) +
  labs(title = "Vertrauen in die Regierung 2017",
       subtitle = "1 = Überhaupt kein Vertrauen, 4 = Sehr viel Vertrauen",
       fill = "",
       caption = "Quelle: European Values Study 2017: Integrated Dataset (EVS 2017). \nGESIS Datenarchiv, Köln. ZA7500 Datenfile Version 1.0.0, doi:10.4232/1.13090") +
  scale_fill_distiller(breaks = c(1, 1.5, 2, 2.5, 3, 3.5, 4), palette = "RdYlGn", direction = 1, limits = c(1.5,3.5))

Im nächsten Plot werden die Antworten auf die Frage visualisiert, wie wichtig ist es für die Befragten ist, in einem Land zu leben, das demokratisch regiert wird.

In [ ]:
ggplot(data = evs_map_2017, aes(x = long, y = lat, group = group, fill = v142)) + 
  geom_polygon(color = "white") + 
  xlim(-25, 47.0) + ylim(35, 70) +
  theme_map() +
  theme(plot.title = element_text(face="bold", size = 14)) +
  labs(title = "Wie wichtig ist es für Sie, in einem Land zu leben, \ndas demokratisch regiert wird?",
       subtitle = "1 = Überhaupt nicht wichtig, 10 = Absolut wichtig",
       fill = "",
       caption = "Quelle: European Values Study 2017: Integrated Dataset (EVS 2017). \nGESIS Datenarchiv, Köln. ZA7500 Datenfile Version 1.0.0, doi:10.4232/1.13090") +
  scale_fill_distiller(breaks = c(7, 7.5, 8, 8.5, 9, 9.5, 10), palette = "RdYlGn", direction = 1, limits = c(7, 10))

Die vierte Beispielgrafik zeigt die Zufriedenheit mit dem politischen System im eigenen Land.

In [ ]:
ggplot(data = evs_map_2017, aes(x = long, y = lat, group = group, fill = v144)) + 
  geom_polygon(color = "white") + 
  xlim(-25, 47.0) + ylim(35, 70) +
  theme_map() +
  theme(plot.title = element_text(face="bold", size = 13)) +
  labs(title = "Wie zufrieden sind Sie damit, wie das politische System \nin Ihrem Land heutzutage funktioniert?",
       subtitle = "1 = Überhaupt nicht zufrieden, 10 = Voll und ganz zufrieden",
       fill = "",
       caption = "Quelle: European Values Study 2017: Integrated Dataset (EVS 2017). \nGESIS Datenarchiv, Köln. ZA7500 Datenfile Version 1.0.0, doi:10.4232/1.13090") +
  scale_fill_distiller(breaks = c(3, 4, 5, 6, 7, 8), palette = "RdYlGn", direction = 1, limits = c(2.5, 8))

## EVS 2008

Für einen Vergleich von mehr Ländern sowie Einblicke in die Entwicklung der Meinungen über die Zeit verwenden wir Daten aus dem vollständigen 2008er EVS-Datensatz. Die Schritte zur Vorbereitung der Visualisierung sind die gleichen wie für die EVS-Daten aus dem Jahr 2017. Wichtig zu beachten ist hierbei, dass sich die Variablennamen zwischen 2008 und 2017 unterscheiden. Zudem wurden die Wichtigkeit von Demokratie sowie die Zufriedenheit mit dem politischen System in der EVS 2008 nicht abgefragt.

In [ ]:
download_dataset(s, doi = "4800", path = "./data", filetype = ".sav", purpose = 6)
unzip("./data/ZA4800_v4-0-0.sav.zip", exdir = "./data")
evs_2008 <- read_sav("data/ZA4800_v4-0-0.sav")
dim(evs_2008)
pol_evs_2008 <- evs_2008 %>% 
  select(country, v214, v222) %>% 
  mutate(country = as_factor(country),
         v214 = recode(as.numeric(v214), "1" = 4,
                       "2" = 3,
                       "3" = 2,
                       "4" = 1),
         v222 = recode(as.numeric(v222), "1" = 4,
                       "2" = 3,
                       "3" = 2,
                       "4" = 1))
pol_country_evs_2008 <- pol_evs_2008 %>% 
  group_by(country) %>% 
  summarize_all(mean, na.rm = T)
pol_country_evs_2008 %>% anti_join(world, by = c("country" = "region"))
world %>% 
    select(region) %>% 
    unique()
pol_country_evs_2008 <- pol_country_evs_2008 %>% 
  mutate(country = recode(country, `Bosnia Herzegovina` = 'Bosnia and Herzegovina',
                          `Slovak Republic` = 'Slovakia',
                          `Russian Federation` = 'Russia',
                          `Great Britain` = 'UK'))
world <- world %>%
  mutate(region = ifelse(region == "UK" & subregion == "Northern Ireland", "Northern Ireland",
                         ifelse(region == "Cyprus" & !is.na(subregion), "Northern Cyprus", region)))
evs_map_2008 <- world %>% 
  left_join(pol_country_evs_2008, by = c("region" = "country"))
pol_country_evs_2008 %>% 
  select(-country) %>% 
  summary()

Plot zum Vertrauen in die EU 2008.

In [ ]:
ggplot(data = evs_map_2008, aes(x = long, y = lat, group = group, fill = v214)) + 
  geom_polygon(color = "white") + 
  xlim(-25, 47.0) + ylim(35, 70) +
  theme_map() +
  theme(plot.title = element_text(face="bold", size = 16)) +
  labs(title = "Vertrauen in die EU 2008",
       subtitle = "1 = Überhaupt kein Vertrauen, 4 = Sehr viel Vertrauen",
       fill = "",
       caption = "Quelle: European Values Study 2008: Integrated Dataset (EVS 2008). \nGESIS Datenarchiv, Köln. ZA4800 Datenfile Version 4.0.0, doi:10.4232/1.12458") +
  scale_fill_distiller(breaks = c(1, 1.5, 2, 2.5, 3, 3.5, 4), palette = "RdYlGn", direction = 1, limits = c(1.5,3.5))

Plot zum Vertrauen in die eigene Regierung 2008.

In [ ]:
ggplot(data = evs_map_2008, aes(x = long, y = lat, group = group, fill = v222)) + 
  geom_polygon(color = "white") + 
  xlim(-25, 47.0) + ylim(35, 70) +
  theme_map() +
  theme(plot.title = element_text(face="bold", size = 16)) +
  labs(title = "Vertrauen in die Regierung 2008",
       subtitle = "1 = Überhaupt kein Vertrauen, 4 = Sehr viel Vertrauen",
       fill = "",
       caption = "Quelle:European Values Study 2008: Integrated Dataset (EVS 2008). \nGESIS Datenarchiv, Köln. ZA4800 Datenfile Version 4.0.0, doi:10.4232/1.12458") +
  scale_fill_distiller(breaks = c(1, 1.5, 2, 2.5, 3, 3.5, 4), palette = "RdYlGn", direction = 1, limits = c(1.5,3.5))

Wenn Sie mit der Arbeit mit/an diesem Notebook komplett durch sind bzw. diese beenden wollen, wählen Sie bitte im Reiter "File" den Punkt "Close and Halt" und klicken dann in der Übersicht mit den Dateien oben rechts den Button "Quit".